<a href="https://colab.research.google.com/github/Gpreethi18/Text-To-Speech-with-emotion/blob/main/Automated_AudioBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('dataset_2.csv')

Happy, Sad, Angry, Fear, Normal, Suprised

In [ ]:
df.head()

**DATA PREPROCESSING**

In [ ]:
df.isnull().sum()

In [ ]:
df.fillna(1, inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df['Emotion'].value_counts()

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

In [ ]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
tokenized = df['Sentence'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
np.array(padded).shape

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ftf = last_hidden_states[0].numpy()

In [ ]:
print(ftf.shape)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
print(features.shape)
print(features[0].shape)
features

In [ ]:
labels = df['Emotion']

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

In [ ]:
lr_clf = LogisticRegression(C=5.263252631578947)

In [ ]:
lr_clf.fit(train_features, train_labels)

In [ ]:
lr_clf.score(test_features, test_labels)

NameError: ignored

In [ ]:
predictions = lr_clf.predict(test_features)

In [ ]:
predictions

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_labels, predictions))

In [ ]:
def transform(df):
  tokenized = df['Sentence'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
  max_len = 0
  for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
  attention_mask = np.where(padded != 0, 1, 0)
  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
  features = last_hidden_states[0][:,0,:].numpy()
  return features

In [ ]:
import pickle
with open('lr_clf_model', 'wb') as files:
    pickle.dump(lr_clf, files)

In [ ]:
import pickle
with open('/content/drive/MyDrive/ED/Pretrained models/lr_clf_model' , 'rb') as f:
    lr_clf_2 = pickle.load(f)

In [ ]:
embed_array = ['__', 'normal_embed', 'angry_embed', 'fear_embed', 'happy_embed', '__', 'sad_embed', 'suprise_embed']

In [ ]:
%tensorflow_version 2.x
import os
from os.path import exists, join, basename, splitext
git_repo_url = 'https://github.com/CorentinJ/Real-Time-Voice-Cloning.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  # install dependencies
  !cd {project_name} && pip install -q -r requirements.txt
  !pip install -q gdown
  !apt-get install -qq libportaudio2
  !pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip

import sys
sys.path.append(project_name)

from IPython.display import display, Audio, clear_output
from IPython.utils import io
import ipywidgets as widgets
import numpy as np
from dl_colab_notebooks.audio import record_audio, upload_audio

from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path

encoder.load_model(Path("/content/drive/MyDrive/ED/Pretrained models/encoder.pt"))
synthesizer = Synthesizer(Path("/content/drive/MyDrive/ED/Pretrained models/synthesizer.pt"))
vocoder.load_model(Path("/content/drive/MyDrive/ED/Pretrained models/vocoder.pt"))

In [ ]:
def synthesize(embed, text):
  print("Synthesizing new audio...")
  #with io.capture_output() as captured:
  specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
  clear_output()
  display(Audio(generated_wav, rate=synthesizer.sample_rate, autoplay=True))


In [ ]:
Sentence_list = ['''I AM happy but climate is bad''']
values = {'Sentence':  Sentence_list}
test_data = pd.DataFrame(values)
test_feature = transform(test_data)

In [ ]:
prediction = lr_clf_2.predict(test_feature)

In [ ]:
prediction

In [ ]:
embed = Path("/content/drive/MyDrive/ED/Pretrained models/" + embed_array[int(prediction[0])])

In [ ]:
with open(embed , 'rb') as f:
    embedding_emo = pickle.load(f)

In [ ]:
synthesize(embedding_emo, Sentence_list[0])

In [ ]:
print("================= SVM =================")
from sklearn import svm

#Create a svm Classifier
lr_clf = svm.SVC(kernel='linear') # Linear Kernel
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)
predictions = lr_clf.predict(test_features)
print(predictions)
from sklearn.metrics import accuracy_score
print(accuracy_score(test_labels, predictions))




In [ ]:
print("=================RandomForestClassifier =================")
from sklearn.ensemble import RandomForestClassifier
lr_clf = RandomForestClassifier(n_estimators=50, criterion="entropy")
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)
predictions = lr_clf.predict(test_features)
print(predictions)
print(accuracy_score(test_labels, predictions))